In [1]:
import os
import plotly.express as px

import matplotlib.pyplot as plt

import basd
from matplotlib import pyplot
import numpy as np
import pandas as pd
import rioxarray as rio
import seaborn as sns
import xarray as xr

#### Reading NetCDF file
Reading in observational precipitation data from 1979-2014 supplied in the data directory.

In [2]:
pr_obs_hist = xr.open_dataset('../basd/data/pr_obs-hist_fine_1979-2014.nc')
pr_sim_hist = xr.open_dataset('../basd/data/pr_sim-hist_coarse_1979-2014.nc')
pr_sim_fut = xr.open_dataset('../basd/data/pr_sim-fut_coarse_2065-2100.nc')

### Running Bias Adjustment
#### Initializing our first `Adjustment` object
Here we are providing the three required data arrays and a variable name 'pr' representing the daily mean precipitation variable. Precipitation is assumed to follow a gamma distribution, and thus requires a lower bound and lower threshold parameter. We also set trend preservation mode to be 'mixed'. Thus, we set the parameter object directly specifying:

* Lower bound = 0
* Lower threshold = 0.0000011574
* Trend preservation = mixed
* Distribution = gamma

Otherwise, we are keeping the default parameter values for the bias adjustment.

In [3]:
params = basd.Parameters(lower_bound=0, lower_threshold=0.0000011574,
                         trend_preservation='mixed', distribution='gamma',
                         if_all_invalid_use=0)
ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut,
                     'pr', params, remap_grid=True )

#### Running Bias Adjustment
Here we run the bias adjustment for just one grid cell. We pass in a tuple with the indexes which we wish to adjust, (0,0), which in this case correspond to the coordinates 53.5 N, 12.5 E.

In [4]:
loc = dict(lat=1, lon=1)
sim_fut_ba_loc = ba.adjust_bias_one_location(loc)

In [5]:
pr_sim_fut_ba = ba.adjust_bias(n_jobs=-1)
#ba.save_adjustment_nc('../../../Documents/pr_sim_fut_ba.nc')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    6.4s remaining:    6.4s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    6.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    6.5s finished


# Statistical Downscaling
We're now going to start to look at downscaling simulated data to observational data resolution using the statistical downscaling algorithm.

In [6]:
sd = basd.Downscaler(pr_obs_hist, pr_sim_fut_ba, 'pr', params)

Latitude and longitude coordinate sequences for each of the input grids

In [11]:
fine_lats = sd.obs_fine.coords['lat'].values
fine_lons = sd.obs_fine.coords['lon'].values
coarse_lats = sd.sim_coarse.coords['lat'].values
coarse_lons = sd.sim_coarse.coords['lon'].values

Finding the downscaling factor. Make sure that the fine cells fit evenly in the coarse cells

In [22]:
assert len(fine_lats) % len(coarse_lats) == 0
assert len(fine_lons) % len(coarse_lons) == 0
f_lat = len(fine_lats) // len(coarse_lats)
f_lon = len(fine_lons) // len(coarse_lons)
assert f_lat > 1 or f_lon > 1, f'No downscaling needed. Observational grid provides no finer resolution'

Sequence of gaps between coordinates. Assert that they are constant and given in the same direction.

In [91]:
dx_lat = np.diff(coarse_lats)
dy_lat = np.diff(fine_lats)
assert (np.all(dx_lat > 0) and np.all(dy_lat > 0)) or (np.all(dx_lat < 0) and np.all(dy_lat < 0)), f'Latitude coords should be monotonic in the same direction'
dx_lon = np.diff(coarse_lons)
dy_lon = np.diff(fine_lons)
assert (np.all(dx_lon > 0) and np.all(dy_lon > 0)) or (np.all(dx_lon < 0) and np.all(dy_lon < 0)), f'Longitude coords should be monotonic in the same direction'
assert np.all(dx_lon == dx_lon[0]) and np.all(dy_lon == dy_lon[0]), f'Resolution should be constant for all longitude'
assert np.all(dx_lat == dx_lat[0]) and np.all(dy_lat == dy_lat[0]), f'Resolution should be constant for all latitude'

Whether longitude wraps around globe. Latitude of course does not work that way.

In [ ]:
circular = {
    'lat': False,
    'lon': np.allclose(coarse_lons[0] - dx_lon[0] + 360 * np.sign(dx_lon[0]), coarse_lons[-1])
}
ascending = {
    'lat': dx_lat[0] > 0,
    'lon': dx_lon[0] > 0
}
scale_factors = {
    'lat': f_lat,
    'lon': f_lon
}

In [97]:
shape = tuple([sd.obs_fine.coords['lat'].size, sd.obs_fine.coords['lon'].size])
lats = sd.obs_fine.coords['lat'].values
weight_by_lat = np.cos(np.deg2rad(lats))

In [101]:
weight_by_lat
lats

array([53.75, 53.25, 52.75, 52.25])